In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

import spacy
from spacy.training import Example
import random

import ahocorasick
spacy.prefer_gpu()

True

In [10]:
# df = pd.read_csv("./Address Elements Extraction Dataset/train.csv")

# df.set_index("id", inplace=True)

# df['POI'] = np.nan
# df['street'] = np.nan
df = pd.read_csv('./split_result.csv')

In [7]:
df_test = pd.read_csv("./Address Elements Extraction Dataset/test.csv")
df_test.set_index("id", inplace=True)

In [11]:
df.head()

,id,raw_address,POI/street,POI,street
0,0,jl kapuk timur delta sili iii lippo cika 11 a cicau cikarang pusat,/jl kapuk timur delta sili iii lippo cika,NaN,jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",/,NaN,NaN
2,2,setu siung 119 rt 5 1 13880 cipayung,/siung,NaN,siung
3,3,"toko dita, kertosono",toko dita/,toko dita,NaN
4,4,jl. orde baru,/jl. orde baru,NaN,jl. orde baru


In [4]:
df_test.head()

,raw_address
id,
0,s. par 53 sidanegara 4 cilacap tengah
1,"angg per, baloi indah kel. lubuk baja"
2,"asma laun, mand imog,"
3,"ud agung rej, raya nga sri wedari karanganyar"
4,"cut mutia, 35 baiturrahman"


In [49]:
model_name = "./mix_model"
model_name = "./poi_model"
# model_name = "./street_model"
model_name = "./filter_model"

In [50]:
spacy.require_gpu()
nlp_model = spacy.load(model_name)
poi_model = spacy.load("./poi_model")

In [54]:
count = 0
for idx, row in df.iloc[520:].iterrows():
#     if  isinstance(row['POI'], float):
    count += 1
    print(f"address: {row['raw_address']}")
    print(f"expected poi: {row['POI']}")
    print(f"expected street: {row['street']}")

    doc_filter = nlp_model(row['raw_address'])
    doc_poi = poi_model(row['raw_address'])
    for ent in doc_filter.ents:
        print("\t **", ent.text, "-", ent.label_)
    
    for ent in doc_poi.ents:
        print("\t ", ent.text, "-", ent.label_)

    print("-" * 50)
    if count == 10:
        break

address: raya banjarn - pur, 98 sokaraja tengah sokaraja
expected poi: nan
expected street: raya banjarnegara - purwok
	 ** raya banjarn - pur - STREET
--------------------------------------------------
address: indo tek, raya indu, pasirsari cikarang selatan
expected poi: indo teknik
expected street: raya indu
	 ** raya indu - STREET
	  indo tek - POI
--------------------------------------------------
address: gang sas ciroyom andir
expected poi: nan
expected street: gang sas
	  gang sas ciroyom - POI
--------------------------------------------------
address: raya kerob 89 kerobokan kuta utara
expected poi: nan
expected street: raya kerob
	 ** raya kerob - STREET
	  kerobokan - POI
--------------------------------------------------
address: war pla, tamalate
expected poi: warnet planet
expected street: nan
	  war - POI
--------------------------------------------------
address: badan pertan nasional kabupaten jayawi, wamena kota wamena
expected poi: badan pertanahan nasional kabupate

In [55]:
submission = []
for idx, row in df_test.iloc[:].iterrows():
    
    doc_filter = nlp_model(row['raw_address'])
    doc_poi = poi_model(row['raw_address'])
    
    tmp = {'id': idx}
    for ent in doc_filter.ents:
        tmp[ent.label_] = ent.text
    for ent in doc_poi.ents:
        tmp["Pmodel"+ent.label_] = ent.text
        
    submission.append(tmp)

In [56]:
submission =pd.DataFrame(submission)
submission['raw_address'] = df_test['raw_address']

In [73]:
submission.head(20)

,id,STREET,POI,PmodelPOI,raw_address
0,0,s. par,sidanegara 4,NaN,s. par 53 sidanegara 4 cilacap tengah
1,1,angg per,NaN,NaN,"angg per, baloi indah kel. lubuk baja"
2,2,mand imog,asma laun,asma laun,"asma laun, mand imog,"
3,3,raya nga sri,ud agung rej,ud agung rej,"ud agung rej, raya nga sri wedari karanganyar"
4,4,cut mutia,NaN,NaN,"cut mutia, 35 baiturrahman"
5,5,perum gar,dapur ala,dapur ala,pem dos dapur ala perum gar no a 12 suka jaya sukarami
6,6,mara,tb. mara,tb. mara,"tb. mara, cisayong"
7,7,NaN,pura taman beji tista,pura taman beji tista,"pura taman beji tista,"
8,8,pam,tk pam,tk pam,"mangkura tk pam, 90113 ujung pandang"
9,9,raya won,NaN,NaN,raya won wonotunggal wonotunggal


In [59]:
street_list =  submission['STREET'].to_list()
poi_list =  submission['POI'].to_list()
PmodelPOI_list =  submission['PmodelPOI'].to_list()

In [71]:
final_p = []
for s, p, pm in zip(street_list, poi_list, PmodelPOI_list):
    if isinstance(pm, float):
        final_p.append(p)
    elif isinstance(p, float):
        if s != pm or s not in pm or pm not in s: 
            final_p.append(pm)
        else:
            final_p.append(p)
    else:
        final_p.append(p)
    
    
print(len(final_p), len(street_list))

50000 50000


In [72]:
submission['POI'] = final_p

In [74]:
submission = submission.fillna("")
combine_lambda = lambda x: '{}/{}'.format(x['POI'], x['STREET'])
submission["POI/street"] = submission.apply(combine_lambda, axis = 1)

In [75]:
submission = pd.DataFrame({'id':submission['id'],
                  'POI/street':submission['POI/street']})
submission.to_csv('./submission_{}_poi.csv'.format(model_name.split("/")[-1]), header=True, index=False)

In [76]:
submission.head(20)

,id,POI/street
0,0,sidanegara 4/s. par
1,1,/angg per
2,2,asma laun/mand imog
3,3,ud agung rej/raya nga sri
4,4,/cut mutia
5,5,dapur ala/perum gar
6,6,tb. mara/mara
7,7,pura taman beji tista/
8,8,tk pam/pam
9,9,/raya won
